In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .appName("MyApp") \
        .config("spark.jars", "/opt/jars/hadoop-aws-3.3.1.jar,/opt/jars/aws-java-sdk-bundle-1.12.375.jar,/opt/jars/delta-spark_2.12-3.2.1.jar,/opt/jars/delta-storage-3.2.1.jar,/opt/jars/delta-kernel-api-3.2.1.jar,/opt/jars/mysql-connector-j-8.3.0.jar") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
        .config("spark.hadoop.fs.s3a.access.key", "NJ3L4SVUPNXQG3Q8G467") \
        .config("spark.hadoop.fs.s3a.secret.key", "xJwXJYnzpsIDlDpFGOPspi8ryChaBJJ94XMFICwj") \
        .config("spark.hadoop.fs.s3a.endpoint", "https://rgw.nau.fccn.pt") \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .getOrCreate()

from pyspark.sql import functions as F

In [8]:
df_auth_userprofile_bronze = spark.read.format("delta").load("s3a://nau-local-analytics-bronze/auth_userprofile")
df_auth_userprofile_bronze.printSchema()
df_auth_userprofile_bronze.show(5)


25/11/13 15:52:56 INFO BlockManagerInfo: Removed broadcast_17_piece0 on jupyter-spark-758b7c86d8-7j77n:32921 in memory (size: 37.1 KiB, free: 1048.6 MiB)
25/11/13 15:52:56 INFO BlockManagerInfo: Removed broadcast_16_piece0 on jupyter-spark-758b7c86d8-7j77n:32921 in memory (size: 164.5 KiB, free: 1048.7 MiB)
25/11/13 15:52:56 INFO BlockManagerInfo: Removed broadcast_18_piece0 on jupyter-spark-758b7c86d8-7j77n:32921 in memory (size: 11.1 KiB, free: 1048.7 MiB)


root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- meta: string (nullable = true)
 |-- courseware: string (nullable = true)
 |-- language: string (nullable = true)
 |-- location: string (nullable = true)
 |-- year_of_birth: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- level_of_education: string (nullable = true)
 |-- mailing_address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- goals: string (nullable = true)
 |-- bio: string (nullable = true)
 |-- profile_image_uploaded_at: timestamp (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- state: string (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)
 |-- source_name: string (nullable = true)



25/11/13 15:52:56 INFO PrepareDeltaScan: DELTA: Filtering files for query
25/11/13 15:52:56 INFO SparkContext: Starting job: $anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128
25/11/13 15:52:56 INFO DAGScheduler: Got job 12 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128) with 50 output partitions
25/11/13 15:52:56 INFO DAGScheduler: Final stage: ResultStage 20 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128)
25/11/13 15:52:56 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 19)
25/11/13 15:52:56 INFO DAGScheduler: Missing parents: List()
25/11/13 15:52:56 INFO DAGScheduler: Submitting ResultStage 20 (MapPartitionsRDD[50] at $anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128), which has no missing parents
25/11/13 15:52:56 INFO MemoryStore: Block broadcast_19 stored as values in memory (estimated size 721.9 KiB, free 1047.6 MiB)
25/11/13 15:52:56 INFO MemoryStore: Block broadcast_19_p

+---+----+----+----------+--------+--------+-------------+------+------------------+---------------+----+-------+-----+----+-------------------------+-------+------------+-----+--------------------+----------------+
| id|name|meta|courseware|language|location|year_of_birth|gender|level_of_education|mailing_address|city|country|goals| bio|profile_image_uploaded_at|user_id|phone_number|state|      ingestion_date|     source_name|
+---+----+----+----------+--------+--------+-------------+------+------------------+---------------+----+-------+-----+----+-------------------------+-------+------------+-----+--------------------+----------------+
|  1|    |    |course.xml|        |        |         NULL|  NULL|              NULL|           NULL|NULL|   NULL| NULL|NULL|                     NULL|      1|        NULL| NULL|2025-11-04 18:27:...|auth_userprofile|
|  2|    |    |course.xml|        |        |         NULL|  NULL|              NULL|           NULL|NULL|   NULL| NULL|NULL|            

25/11/13 15:52:59 INFO Executor: Finished task 0.0 in stage 21.0 (TID 307). 2215 bytes result sent to driver
25/11/13 15:52:59 INFO TaskSetManager: Finished task 0.0 in stage 21.0 (TID 307) in 588 ms on jupyter-spark-758b7c86d8-7j77n (executor driver) (1/1)
25/11/13 15:52:59 INFO TaskSchedulerImpl: Removed TaskSet 21.0, whose tasks have all completed, from pool 
25/11/13 15:52:59 INFO DAGScheduler: ResultStage 21 (showString at NativeMethodAccessorImpl.java:0) finished in 0.600 s
25/11/13 15:52:59 INFO DAGScheduler: Job 13 is finished. Cancelling potential speculative or zombie tasks for this job
25/11/13 15:52:59 INFO TaskSchedulerImpl: Killing all running tasks in stage 21: Stage finished
25/11/13 15:52:59 INFO DAGScheduler: Job 13 finished: showString at NativeMethodAccessorImpl.java:0, took 0.608187 s


In [13]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException

silver_profile_path = "s3a://nau-local-analytics-silver/dim_user_profile"
dim_profile_table_name = "dim_user_profile"

PROFILE_BUSINESS_COLUMNS = [
    "profile_id",
    "user_id",
    "year_of_birth",
    "age_band",
    "gender",
    "country",
    "city",
    "location",
    "level_of_education",
    "language",
    "courseware",
    "phone_number",
    "state",
    "ingestion_date",
    "source_name",
]

# ============================================================
# 1. Check if Silver Delta table already exists
# ============================================================
try:
    df_dim_profile_current = (
        spark.read
             .format("delta")
             .load(silver_profile_path)
             .select("profile_id", "profile_sk")
    )
    dim_profile_exists = True
    print(">> [dim_user_profile] Existing Silver Delta table found.")
except AnalysisException:
    dim_profile_exists = False
    df_dim_profile_current = None
    print(">> [dim_user_profile] Silver Delta table does not exist yet (first load).")

# ============================================================
# 2. Build df_user_profile_silver with stable profile_sk
# ============================================================
if dim_profile_exists:
    max_sk = df_dim_profile_current.agg(F.max("profile_sk")).collect()[0][0]
    max_sk = max_sk if max_sk is not None else 0

    df_joined_profile = (
        df_user_profile_source.alias("s")
        .join(
            df_dim_profile_current.alias("t"),
            on="profile_id",
            how="left"
        )
    )

    # Existing profiles -> reuse profile_sk
    df_profile_existing = (
        df_joined_profile
        .filter(F.col("t.profile_sk").isNotNull())
        .select(
            F.col("t.profile_sk").alias("profile_sk"),
            *[F.col(f"s.{c}").alias(c) for c in PROFILE_BUSINESS_COLUMNS]
        )
    )

    # New profiles -> generate new SKs
    df_profile_new = df_joined_profile.filter(F.col("t.profile_sk").isNull())

    w_new_profiles = Window.orderBy(F.col("s.profile_id"))

    df_profile_new_with_sk = (
        df_profile_new
        .withColumn(
            "profile_sk",
            F.row_number().over(w_new_profiles) + F.lit(max_sk)
        )
        .select(
            F.col("profile_sk"),
            *[F.col(f"s.{c}").alias(c) for c in PROFILE_BUSINESS_COLUMNS]
        )
    )

    df_user_profile_silver = df_profile_existing.unionByName(df_profile_new_with_sk)

else:
    w_initial_profiles = Window.orderBy("profile_id")

    df_user_profile_silver = (
        df_user_profile_source
        .withColumn("profile_sk", F.row_number().over(w_initial_profiles))
        .select(
            "profile_sk",
            *PROFILE_BUSINESS_COLUMNS
        )
    )

print(">> [dim_user_profile] Preview of df_user_profile_silver (with profile_sk):")
df_user_profile_silver.printSchema()
df_user_profile_silver.show(10, truncate=False)

# ============================================================
# 3. Upsert into Silver using Delta MERGE (SQL)
# ============================================================
if dim_profile_exists:
    print(">> [dim_user_profile] Registering staging view and running MERGE...")

    df_user_profile_silver.createOrReplaceTempView("stg_dim_user_profile")

    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {dim_profile_table_name}
        USING delta
        LOCATION '{silver_profile_path}'
    """)

    spark.sql(f"""
        MERGE INTO {dim_profile_table_name} AS t
        USING stg_dim_user_profile AS s
        ON t.profile_id = s.profile_id
        WHEN MATCHED THEN UPDATE SET
            t.user_id            = s.user_id,
            t.year_of_birth      = s.year_of_birth,
            t.age_band           = s.age_band,
            t.gender             = s.gender,
            t.country            = s.country,
            t.city               = s.city,
            t.location           = s.location,
            t.level_of_education = s.level_of_education,
            t.language           = s.language,
            t.courseware         = s.courseware,
            t.phone_number       = s.phone_number,
            t.state              = s.state,
            t.ingestion_date     = s.ingestion_date,
            t.source_name        = s.source_name
        WHEN NOT MATCHED THEN INSERT (
            profile_sk,
            profile_id,
            user_id,
            year_of_birth,
            age_band,
            gender,
            country,
            city,
            location,
            level_of_education,
            language,
            courseware,
            phone_number,
            state,
            ingestion_date,
            source_name
        )
        VALUES (
            s.profile_sk,
            s.profile_id,
            s.user_id,
            s.year_of_birth,
            s.age_band,
            s.gender,
            s.country,
            s.city,
            s.location,
            s.level_of_education,
            s.language,
            s.courseware,
            s.phone_number,
            s.state,
            s.ingestion_date,
            s.source_name
        )
    """)
else:
    print(">> [dim_user_profile] Writing initial Silver Delta table (overwrite).")

    (
        df_user_profile_silver
            .write
            .format("delta")
            .mode("overwrite")
            .option("overwriteSchema", "true")
            .save(silver_profile_path)
    )

    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {dim_profile_table_name}
        USING delta
        LOCATION '{silver_profile_path}'
    """)

print(">> [dim_user_profile] Load/MERGE finished.")


25/11/13 16:15:14 INFO DelegatingLogStore: LogStore `LogStoreAdapter(io.delta.storage.S3SingleDriverLogStore)` is used for scheme `s3a`
25/11/13 16:15:14 INFO DeltaLog: Creating initial snapshot without metadata, because the directory is empty
25/11/13 16:15:14 INFO InitialSnapshot: [tableId=153cb307-46e6-48c9-8558-36f05025fa14] Created snapshot InitialSnapshot(path=s3a://nau-local-analytics-silver/dim_user_profile/_delta_log, version=-1, metadata=Metadata(8331b31f-065d-44ac-ab7f-d88f606cf74a,null,null,Format(parquet,Map()),null,List(),Map(),Some(1763050514238)), logSegment=LogSegment(s3a://nau-local-analytics-silver/dim_user_profile/_delta_log,-1,List(),org.apache.spark.sql.delta.EmptyCheckpointProvider$@2b65c581,-1), checksumOpt=None)


>> [dim_user_profile] Silver Delta table does not exist yet (first load).
>> [dim_user_profile] Preview of df_user_profile_silver (with profile_sk):
root
 |-- profile_sk: integer (nullable = false)
 |-- profile_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- year_of_birth: integer (nullable = true)
 |-- age_band: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- location: string (nullable = true)
 |-- level_of_education: string (nullable = true)
 |-- language: string (nullable = true)
 |-- courseware: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- state: string (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)
 |-- source_name: string (nullable = true)



25/11/13 16:15:14 INFO PrepareDeltaScan: DELTA: Filtering files for query
25/11/13 16:15:15 INFO SparkContext: Starting job: $anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128
25/11/13 16:15:15 INFO DAGScheduler: Got job 16 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128) with 50 output partitions
25/11/13 16:15:15 INFO DAGScheduler: Final stage: ResultStage 26 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128)
25/11/13 16:15:15 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 25)
25/11/13 16:15:15 INFO DAGScheduler: Missing parents: List()
25/11/13 16:15:15 INFO DAGScheduler: Submitting ResultStage 26 (MapPartitionsRDD[62] at $anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128), which has no missing parents
25/11/13 16:15:15 INFO MemoryStore: Block broadcast_25 stored as values in memory (estimated size 684.9 KiB, free 1047.7 MiB)
25/11/13 16:15:15 INFO MemoryStore: Block broadcast_25_p

+----------+----------+-------+-------------+--------+------+-------+----+--------+------------------+--------+----------+------------+-----+--------------------------+----------------+
|profile_sk|profile_id|user_id|year_of_birth|age_band|gender|country|city|location|level_of_education|language|courseware|phone_number|state|ingestion_date            |source_name     |
+----------+----------+-------+-------------+--------+------+-------+----+--------+------------------+--------+----------+------------+-----+--------------------------+----------------+
|1         |1         |1      |NULL         |NULL    |NULL  |NULL   |NULL|        |NULL              |        |course.xml|NULL        |NULL |2025-11-04 18:27:24.809743|auth_userprofile|
|2         |2         |2      |NULL         |NULL    |NULL  |NULL   |NULL|        |NULL              |        |course.xml|NULL        |NULL |2025-11-04 18:27:24.809743|auth_userprofile|
|3         |3         |3      |NULL         |NULL    |NULL  |NULL   |N

25/11/13 16:15:20 INFO DeltaLog: Creating initial snapshot without metadata, because the directory is empty
25/11/13 16:15:20 INFO InitialSnapshot: [tableId=8331b31f-065d-44ac-ab7f-d88f606cf74a] Created snapshot InitialSnapshot(path=s3a://nau-local-analytics-silver/dim_user_profile/_delta_log, version=-1, metadata=Metadata(8b0029bd-d62f-406f-9cab-5818b29ae118,null,null,Format(parquet,Map()),null,List(),Map(),Some(1763050520311)), logSegment=LogSegment(s3a://nau-local-analytics-silver/dim_user_profile/_delta_log,-1,List(),org.apache.spark.sql.delta.EmptyCheckpointProvider$@2b65c581,-1), checksumOpt=None)
25/11/13 16:15:20 INFO BlockManagerInfo: Removed broadcast_27_piece0 on jupyter-spark-758b7c86d8-7j77n:32921 in memory (size: 20.4 KiB, free: 1048.5 MiB)
25/11/13 16:15:20 INFO BlockManagerInfo: Removed broadcast_28_piece0 on jupyter-spark-758b7c86d8-7j77n:32921 in memory (size: 21.0 KiB, free: 1048.5 MiB)
25/11/13 16:15:20 INFO OptimisticTransaction: [tableId=8b0029bd,txnId=e34d9523] U

>> [dim_user_profile] Load/MERGE finished.


25/11/13 16:15:39 INFO SparkContext: Starting job: $anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128
25/11/13 16:15:39 INFO DAGScheduler: Got job 23 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128) with 1 output partitions
25/11/13 16:15:39 INFO DAGScheduler: Final stage: ResultStage 35 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128)
25/11/13 16:15:39 INFO DAGScheduler: Parents of final stage: List()
25/11/13 16:15:39 INFO DAGScheduler: Missing parents: List()
25/11/13 16:15:39 INFO DAGScheduler: Submitting ResultStage 35 (MapPartitionsRDD[95] at $anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128), which has no missing parents
25/11/13 16:15:39 INFO MemoryStore: Block broadcast_36 stored as values in memory (estimated size 55.2 KiB, free 1047.8 MiB)
25/11/13 16:15:39 INFO MemoryStore: Block broadcast_36_piece0 stored as bytes in memory (estimated size 17.1 KiB, free 1047.8 MiB)
25/11/13 16:15:39 I

In [14]:
silver_profile_path = "s3a://nau-local-analytics-silver/dim_user_profile"
dim_profile_table_name = "dim_user_profile"

print(">> [dim_user_profile] Reading Silver Delta table from path:")
print(f"   {silver_profile_path}\n")

df_dim_user_profile = (
    spark.read
         .format("delta")
         .load(silver_profile_path)
)

print(">> [dim_user_profile] Schema:")
df_dim_user_profile.printSchema()

print(">> [dim_user_profile] Sample rows:")
df_dim_user_profile.orderBy("profile_sk").show(10, truncate=False)

total_rows = df_dim_user_profile.count()
distinct_profiles = df_dim_user_profile.select("profile_id").distinct().count()
distinct_users = df_dim_user_profile.select("user_id").distinct().count()

print(f">> [dim_user_profile] Total rows in Silver: {total_rows}")
print(f">> [dim_user_profile] Distinct profile_id in Silver: {distinct_profiles}")
print(f">> [dim_user_profile] Distinct user_id in Silver: {distinct_users}")

print("\n>> [dim_user_profile] Checking if table is available in the catalog (if created):")
spark.sql(f"SHOW TABLES LIKE '{dim_profile_table_name}'").show(truncate=False)

print("\n>> [dim_user_profile] Preview via SQL (if table is registered):")
spark.sql(f"SELECT * FROM {dim_profile_table_name} ORDER BY profile_sk LIMIT 10").show(truncate=False)


>> [dim_user_profile] Reading Silver Delta table from path:
   s3a://nau-local-analytics-silver/dim_user_profile

>> [dim_user_profile] Schema:
root
 |-- profile_sk: integer (nullable = true)
 |-- profile_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- year_of_birth: integer (nullable = true)
 |-- age_band: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- location: string (nullable = true)
 |-- level_of_education: string (nullable = true)
 |-- language: string (nullable = true)
 |-- courseware: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- state: string (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)
 |-- source_name: string (nullable = true)

>> [dim_user_profile] Sample rows:


25/11/13 16:17:30 INFO PrepareDeltaScan: DELTA: Filtering files for query
25/11/13 16:17:30 INFO Snapshot: [tableId=cecff036-318c-4ebc-be9d-1081081bfefa] DELTA: Compute snapshot for version: 0
25/11/13 16:17:30 INFO MemoryStore: Block broadcast_37 stored as values in memory (estimated size 202.5 KiB, free 1047.8 MiB)
25/11/13 16:17:30 INFO BlockManagerInfo: Removed broadcast_35_piece0 on jupyter-spark-758b7c86d8-7j77n:32921 in memory (size: 35.5 KiB, free: 1048.7 MiB)
25/11/13 16:17:30 INFO BlockManagerInfo: Removed broadcast_36_piece0 on jupyter-spark-758b7c86d8-7j77n:32921 in memory (size: 17.1 KiB, free: 1048.7 MiB)
25/11/13 16:17:30 INFO MemoryStore: Block broadcast_37_piece0 stored as bytes in memory (estimated size 35.5 KiB, free 1048.1 MiB)
25/11/13 16:17:30 INFO BlockManagerInfo: Added broadcast_37_piece0 in memory on jupyter-spark-758b7c86d8-7j77n:32921 (size: 35.5 KiB, free: 1048.7 MiB)
25/11/13 16:17:30 INFO SparkContext: Created broadcast 37 from $anonfun$recordDeltaOperati

+----------+----------+-------+-------------+--------+------+-------+----+--------+------------------+--------+----------+------------+-----+--------------------------+----------------+
|profile_sk|profile_id|user_id|year_of_birth|age_band|gender|country|city|location|level_of_education|language|courseware|phone_number|state|ingestion_date            |source_name     |
+----------+----------+-------+-------------+--------+------+-------+----+--------+------------------+--------+----------+------------+-----+--------------------------+----------------+
|1         |1         |1      |NULL         |NULL    |NULL  |NULL   |NULL|        |NULL              |        |course.xml|NULL        |NULL |2025-11-04 18:27:24.809743|auth_userprofile|
|2         |2         |2      |NULL         |NULL    |NULL  |NULL   |NULL|        |NULL              |        |course.xml|NULL        |NULL |2025-11-04 18:27:24.809743|auth_userprofile|
|3         |3         |3      |NULL         |NULL    |NULL  |NULL   |N

25/11/13 16:17:42 INFO BlockManagerInfo: Removed broadcast_43_piece0 on jupyter-spark-758b7c86d8-7j77n:32921 in memory (size: 36.9 KiB, free: 1048.6 MiB)
25/11/13 16:17:42 INFO BlockManagerInfo: Removed broadcast_44_piece0 on jupyter-spark-758b7c86d8-7j77n:32921 in memory (size: 10.4 KiB, free: 1048.7 MiB)
25/11/13 16:17:42 INFO CodeGenerator: Code generated in 37.337245 ms
25/11/13 16:17:42 INFO DAGScheduler: Registering RDD 125 (count at NativeMethodAccessorImpl.java:0) as input to shuffle 6
25/11/13 16:17:42 INFO DAGScheduler: Got map stage job 29 (count at NativeMethodAccessorImpl.java:0) with 50 output partitions
25/11/13 16:17:42 INFO DAGScheduler: Final stage: ShuffleMapStage 46 (count at NativeMethodAccessorImpl.java:0)
25/11/13 16:17:42 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 45)
25/11/13 16:17:42 INFO DAGScheduler: Missing parents: List()
25/11/13 16:17:42 INFO DAGScheduler: Submitting ShuffleMapStage 46 (MapPartitionsRDD[125] at count at NativeMethodA

>> [dim_user_profile] Total rows in Silver: 494033
>> [dim_user_profile] Distinct profile_id in Silver: 494033
>> [dim_user_profile] Distinct user_id in Silver: 494033

>> [dim_user_profile] Checking if table is available in the catalog (if created):


25/11/13 16:17:56 INFO CodeGenerator: Code generated in 39.117776 ms
25/11/13 16:17:56 INFO CodeGenerator: Code generated in 8.684317 ms
25/11/13 16:17:56 INFO CodeGenerator: Code generated in 8.88097 ms


+---------+----------------+-----------+
|namespace|tableName       |isTemporary|
+---------+----------------+-----------+
|default  |dim_user_profile|false      |
+---------+----------------+-----------+


>> [dim_user_profile] Preview via SQL (if table is registered):


25/11/13 16:17:56 INFO PrepareDeltaScan: DELTA: Filtering files for query
25/11/13 16:17:56 INFO BlockManagerInfo: Removed broadcast_55_piece0 on jupyter-spark-758b7c86d8-7j77n:32921 in memory (size: 21.9 KiB, free: 1048.6 MiB)
25/11/13 16:17:56 INFO BlockManagerInfo: Removed broadcast_53_piece0 on jupyter-spark-758b7c86d8-7j77n:32921 in memory (size: 36.1 KiB, free: 1048.6 MiB)
25/11/13 16:17:56 INFO SparkContext: Starting job: $anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128
25/11/13 16:17:56 INFO DAGScheduler: Got job 39 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128) with 50 output partitions
25/11/13 16:17:56 INFO DAGScheduler: Final stage: ResultStage 67 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128)
25/11/13 16:17:56 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 66)
25/11/13 16:17:56 INFO DAGScheduler: Missing parents: List()
25/11/13 16:17:56 INFO DAGScheduler: Submitting ResultStage 67 (

+----------+----------+-------+-------------+--------+------+-------+----+--------+------------------+--------+----------+------------+-----+--------------------------+----------------+
|profile_sk|profile_id|user_id|year_of_birth|age_band|gender|country|city|location|level_of_education|language|courseware|phone_number|state|ingestion_date            |source_name     |
+----------+----------+-------+-------------+--------+------+-------+----+--------+------------------+--------+----------+------------+-----+--------------------------+----------------+
|1         |1         |1      |NULL         |NULL    |NULL  |NULL   |NULL|        |NULL              |        |course.xml|NULL        |NULL |2025-11-04 18:27:24.809743|auth_userprofile|
|2         |2         |2      |NULL         |NULL    |NULL  |NULL   |NULL|        |NULL              |        |course.xml|NULL        |NULL |2025-11-04 18:27:24.809743|auth_userprofile|
|3         |3         |3      |NULL         |NULL    |NULL  |NULL   |N

25/11/13 16:17:59 INFO Executor: Finished task 0.0 in stage 68.0 (TID 782). 3682 bytes result sent to driver
25/11/13 16:17:59 INFO TaskSetManager: Finished task 0.0 in stage 68.0 (TID 782) in 1562 ms on jupyter-spark-758b7c86d8-7j77n (executor driver) (2/2)
25/11/13 16:17:59 INFO TaskSchedulerImpl: Removed TaskSet 68.0, whose tasks have all completed, from pool 
25/11/13 16:18:00 INFO DAGScheduler: ResultStage 68 (showString at NativeMethodAccessorImpl.java:0) finished in 1.608 s
25/11/13 16:18:00 INFO DAGScheduler: Job 40 is finished. Cancelling potential speculative or zombie tasks for this job
25/11/13 16:18:00 INFO TaskSchedulerImpl: Killing all running tasks in stage 68: Stage finished
25/11/13 16:18:00 INFO DAGScheduler: Job 40 finished: showString at NativeMethodAccessorImpl.java:0, took 1.614643 s
25/11/13 16:32:44 INFO BlockManagerInfo: Removed broadcast_58_piece0 on jupyter-spark-758b7c86d8-7j77n:32921 in memory (size: 36.9 KiB, free: 1048.6 MiB)
25/11/13 16:32:44 INFO Block